## Carga de librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

---
## 1. Yahoo Finance
### 1.1. Spot OHLCV
#### 1.1.1. Velas diarias

In [19]:
tickers = ['SPY', 'QQQ', 'AAPL', 'NVDA', 'TSLA']

spy_daily = yf.download(tickers[0], 
                        start='2020-01-01', 
                        end='2025-07-01',
                        interval='1d',
                        auto_adjust=True,
                        progress=False
                        )

spy_daily.head()


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2020-01-02,299.406433,299.424883,297.249847,298.180692,59151200
2020-01-03,297.139313,298.272915,295.931994,295.987288,77709700
2020-01-06,298.272858,298.355800,295.249920,295.369735,55653900
2020-01-07,297.434235,298.180744,296.982622,297.701484,40496400
2020-01-08,299.019379,300.245120,297.378898,297.627725,68296000


#### 1.1.2. Intradía
Yahoo pone límites segúna la resolución:
- intervalo = 1min -> ventana máx = 7 días
- intervalo = 60min -> ventana máx = 730 días ~2años

In [18]:
spy_intraday = yf.download(tickers[0],
                           period='730d',
                           interval='60m',
                           auto_adjust=True,
                           prepost=True
                           )
spy_intraday.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Datetime,,,,,
2022-08-12 08:00:00+00:00,422.380,422.4000,421.52,422.40,0
2022-08-12 09:00:00+00:00,421.940,422.6500,421.94,422.38,0
2022-08-12 10:00:00+00:00,421.570,422.4500,421.42,421.95,0
2022-08-12 11:00:00+00:00,421.330,421.7500,420.96,421.63,0
2022-08-12 12:00:00+00:00,421.755,422.5304,420.71,421.30,0


### 1.1. Cadena de opciones EOD
Queremos precio, volumen y OI

In [20]:
spy, qqq, aapl, nvda, tsla = [yf.Ticker(ticker) for ticker in tickers]

expirations = spy.options
chain = spy.option_chain(expirations[0])

calls = chain.calls[['contractSymbol','strike', 'lastPrice',
                     'bid', 'ask', 'volume', 'openInterest']]

puts = chain.puts[['contractSymbol','strike', 'lastPrice',
                   'bid', 'ask', 'volume', 'openInterest']] 

calls.head()

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest
0,SPY250711C00375000,375.0,249.91,248.76,250.58,1.0,40
1,SPY250711C00380000,380.0,222.31,243.77,245.53,15.0,0
2,SPY250711C00400000,400.0,225.78,223.76,225.54,1.0,3
3,SPY250711C00410000,410.0,215.84,213.78,215.60,2.0,1
4,SPY250711C00415000,415.0,186.70,0.00,0.00,NaN,0


### Ejecución de CLI
Se puede ejecutar yahoo.py por la terminar escribiendo:
- ` python src/data_ingest/yahoo.py --ticker SPY --spot`
- ` python src/data_ingest/yahoo.py --ticker SPY --options`

Pero también se puede desde el notebook

## 2. IBKR
Usaré *ib_insysn* e *IB Gateway* para automatizarlo y que descargue datos.


## 3. Polygon.io
Principalmente para datos de opciones. Usaremos la REST API ya que nos vale con datos estáticos. Si quisieramos datos live, también tiene un WebSocket API.

In [13]:
from polygon import RESTClient
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.getenv("POLYGON_API_KEY")

client = RESTClient(API_KEY)
status = client.get_market_status()
print(status)

MarketStatus(after_hours=False, currencies=MarketCurrencies(crypto='open', fx='closed'), early_hours=False, exchanges=MarketExchanges(nasdaq='closed', nyse='closed', otc='closed'), indicesGroups=MarketIndices(s_and_p='closed', societe_generale='closed', cgi='closed', msci='closed', ftse_russell='closed', mstar='closed', mstarc='closed', cccy='open', nasdaq='closed', dow_jones='closed'), market='closed', server_time='2025-07-26T13:24:35-04:00')


In [8]:
options_chain = []
for o in client.list_snapshot_options_chain(
    "HCP",
    params={
        "expiration_date.get": "2024-03-16",
        "strike_price.gte": 29,
        "strike_price.lte": 30,
    }
):
    options_chain.append(o)

print(options_chain)
print(len(options_chain))

[]
0
